In [2]:
import os
os.chdir("..")
import numpy as np
from data.loader import load_training, load_validation, n_freq, n_time, n_classes

In [5]:
load_training()

FileNotFoundError: [Errno 2] No such file or directory: 'data/training.npy'

In [4]:
X, y = load_training()
X_val, y_val = load_validation()
#y_binary = y == 1

FileNotFoundError: [Errno 2] No such file or directory: 'data/training.npy'

In [4]:
X.shape

(33849, 32, 96)

In [6]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import log_loss, f1_score

In [7]:
clf = DummyClassifier(strategy="most_frequent")
clf.fit(X, y)


DummyClassifier(strategy='most_frequent')

In [8]:
# Get cross-entropy loss
log_loss(y, clf.predict_proba(X))

17.412207752632092

In [9]:
clf.score(X, y)

0.516913350468256

In [16]:
# Get F1 score
from sklearn.metrics import f1_score, accuracy_score

f1_score(y, clf.predict(X), average="macro")

ValueError: Found array with dim 3. MLPClassifier expected <= 2.

In [11]:
# Lets use linear regression as a baseline
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(
    multi_class="multinomial",
)

clf.fit(X.reshape(-1, n_freq * n_time), y)

/home/annareisz/miniconda3/envs/oticon/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(multi_class='multinomial')

In [12]:
# Get cross-entropy loss
log_loss(y, clf.predict_proba(X.reshape(-1, n_freq * n_time)))

0.9769830464173013

In [13]:
clf.score(X.reshape(-1, n_freq * n_time), y)

0.6291470944488758

In [14]:
f1_score(y, clf.predict(X.reshape(-1, n_freq * n_time)), average="macro")

0.33517882184586456

In [15]:
# Lets use a simple neural network as a baseline
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(
    hidden_layer_sizes=(128, 128),
    max_iter=1000,
    early_stopping=True,
    validation_fraction=0.2,
    random_state=133742069,
)

clf.fit(X.reshape(-1, n_freq * n_time), y)

MLPClassifier(early_stopping=True, hidden_layer_sizes=(128, 128), max_iter=1000,
              random_state=133742069, validation_fraction=0.2)

In [17]:
# Get cross-entropy loss
log_loss(y, clf.predict_proba(X.reshape(-1, n_freq * n_time)))

0.780942825992124

In [18]:
# Get accuracy
clf.score(X.reshape(-1, n_freq * n_time), y)

0.7139058760967828

In [19]:
# Get F1 score
from sklearn.metrics import f1_score

f1_score(y, clf.predict(X.reshape(-1, n_freq * n_time)), average="macro")

0.49219152433719404

In [20]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import TensorDataset, DataLoader

# Convert to tensors
X_tensor = t.from_numpy(X).float()
y_tensor = t.from_numpy(y == 1).int() # Convert to binary, 1 = music

In [63]:
categories_dict = {0: 'Other', 1: 'Music', 2: 'Human voice', 3: 'Engine sounds', 4: 'Alarm Sounds'}import torch.nn as nn

# Tensorflow
# ann = models.Sequential()
# ann.add(lyers.Conv2D(32, (3, 3), activation='relu', input_shape=(*x[0].shape, 1)))
# ann.add(layers.MaxPooling2D((2, 2)))
# ann.add(layers.Dropout(0.2))
# ann.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(*x[0].shape, 1)))
# ann.add(layers.MaxPooling2D((2, 2)))
# ann.add(layers.Dropout(0.2))
# ann.add(layers.Flatten())
# ann.add(layers.Dense(64, activation='relu'))
# ann.add(layers.Dropout(0.2))
# ann.add(layers.Dense(32, activation='relu'))
# ann.add(layers.Dense(1, activation='sigmoid'))

# Torchimport torch.nn as nn

class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0))
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
        self.drop1 = nn.Dropout(p=0.2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(0, 0))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(0, 0))
        self.drop2 = nn.Dropout(p=0.2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(4224, 64)
        self.drop3 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.pool1(x)
        x = self.drop1(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.pool2(x)
        x = self.drop2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.drop3(x)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
        
ann = ANN()



In [64]:
X_tensor.shape, y_tensor.shape

(torch.Size([33849, 32, 96]), torch.Size([33849]))

In [67]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(ann.parameters())

train_loader = DataLoader(TensorDataset(X_tensor, y_tensor), batch_size=64, shuffle=True)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = ann(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

[1,   100] loss: 0.648
[1,   200] loss: 0.445
[1,   300] loss: 0.433
[1,   400] loss: 0.409
[1,   500] loss: 0.393
[2,   100] loss: 0.354
[2,   200] loss: 0.343
[2,   300] loss: 0.351
[2,   400] loss: 0.339
[2,   500] loss: 0.324
[3,   100] loss: 0.319
[3,   200] loss: 0.303
[3,   300] loss: 0.299
[3,   400] loss: 0.285
[3,   500] loss: 0.297
[4,   100] loss: 0.269
[4,   200] loss: 0.277
[4,   300] loss: 0.280
[4,   400] loss: 0.267
[4,   500] loss: 0.285
[5,   100] loss: 0.265
[5,   200] loss: 0.249
[5,   300] loss: 0.271
[5,   400] loss: 0.246
[5,   500] loss: 0.264
[6,   100] loss: 0.248
[6,   200] loss: 0.247
[6,   300] loss: 0.237
[6,   400] loss: 0.230
[6,   500] loss: 0.241
[7,   100] loss: 0.224
[7,   200] loss: 0.237
[7,   300] loss: 0.225
[7,   400] loss: 0.236
[7,   500] loss: 0.227
[8,   100] loss: 0.228
[8,   200] loss: 0.218
[8,   300] loss: 0.227
[8,   400] loss: 0.213
[8,   500] loss: 0.217
[9,   100] loss: 0.197
[9,   200] loss: 0.209
[9,   300] loss: 0.196
[9,   400] 

In [68]:
# Gett F1 score
from sklearn.metrics import f1_score

f1_score(y, ann(X_tensor).detach().numpy() > 0.5, average="macro")

In [ ]:
# Get accuracy
from sklearn.metrics import accuracy_score

accuracy_score(y, ann(X_tensor).detach().numpy() > 0.5)